# <b></bApplication>Application to queuing models</b>

<center>
<table>
<tr><td>$\lambda$</td><td>Average arrival rate (average number of arrivals per unit of time)</td></tr>
<tr><td>$\mu$</td><td>Average service rate (average number of customers completing the service per unit of time)</td></tr>
    
<tr><td>$\pi_n$</td><td>Probability of having n clients in the system.</td></tr>
<tr><td>L</td><td>Average number of customers in the station (waiting customer + customers being served)</td></tr>
<tr><td>L$_q$</td><td>Average number of customers in the queue (excluding customers being served)</td></tr>
<tr><td>W</td><td>Average customer response time (between entering and leaving the station)</td></tr>
<tr><td>W$_q$</td><td>Average waiting time in the queue.</td></tr>
<tr><td>$\rho$</td><td>Utilization factor. i.e the fraction of the average time during which individual servers are busy.</td></tr>
</table>
</center>

$$ \rho=\frac{B}{B+I}=\frac{\lambda}{s\mu} $$

$$\pi_{n} = \frac{s^{min(n, s)}}{min(n, s)!} \rho^n \pi_0  \quad \text{ for } 1 \le n \le s$$

$$L=\lambda \times W$$

$$L_q =  L - (1 - \pi_0) = L - \rho = \lambda \times W_q$$

$$W = W_q + \frac{1}{\mu}  \qquad (\frac{1}{\mu} \textit{is the average service time)}$$




In [1]:
#Code501.py

from math import factorial

In [2]:
# Queueing Class
class Queueing(object):

    # constructor
    def __init__(self, model='MM1',  A='M' ,B='M' ,C=1, K=-1, P=None, D='FIFO', laws =None):
        self.model =  model

        self.s, self.k = C, K
        self.lamda, self.mu = A['params']['lambda'], B['params']['mu']
        self.rho = 1.0 * self.lamda/(self.s*self.mu)

        self.p0 = laws[self.model]['p0'](rho1=self.rho, s=self.s , k=self.k)
        self.pk = self.pn(self.k) if self.k > 0 else 0;
        self.lamdae = (1 - ( 0 if self.k == -1 else self.pk)) * self.lamda
        self.rhoe = 1.0 * self.lamdae/(self.s* self.mu)

        #
        self.Lq = laws[self.model]['Lq'](rho1=self.rhoe, s=self.s , k=self.k, p0=self.p0)
        self.L  = self.Lq + self.s * self.rhoe
        self.W = lambda q : (self.L if q == '' else self.Lq)/self.lamdae

    # isErgodic
    def isErgodic(self):
        return self.rho < 1

    # getStationaryProb
    def pn(self,n):
        pn0 = lambda n: self.s**min(self.s,n)/factorial(min(self.s,n)) * self.rho**n * self.p0
        if n == 0 : return self.p0
        if self.k != -1 and n > self.k  : return 0
        return pn0(n)

    # show perforamnces
    def test(self):
        print('==================' + self.model + ' =======================')
        print("p0* :{:.5f}".format(self.p0))
        print("p1* :{:.5f}".format(self.pn(1)))
        print("Lq  :{:.2f}".format(self.Lq))
        print("L   :{:.2f}".format(self.L))
        print("W   :{:.2f}".format(self.W('')))
        print("Wq  :{:.2f}".format(self.W('q')))

# M/M/1 system
$$
\pi_0 =1-\rho  \\
$$

$$
 L_q=\frac{\rho}{1-\rho} -\rho =\frac{\lambda}{\mu-\lambda} - \rho =  \frac{\rho \pi_0}{(1-\rho)^2} - \rho
 $$

In [3]:
#Code502.py
#from Code501 import Queueing

# getMM1
def getMM1(lamda, mu):
    #
    laws ={
    'MM1' : {
        'p0' : lambda **p: (1 - p['rho1']),                          # p0 = 1 - rho,
        'Lq' : lambda **p: p['rho1'] / (1 - p['rho1']) - p['rho1'],  # Lq = L - rho
        },
    }

    #=========================================================================
    # Tests
    qs = Queueing(
            model = 'MM1', 
            A ={'D':'Pois' , 'params': { 'lambda': lamda}},
            B ={'D':'Expo' , 'params': { 'mu': mu}},
            laws = laws)
    qs.test()


if __name__ == "__main__":  getMM1(4,6)
# ==================MM1 =======================
# p0* :0.33333
# p1* :0.22222
# Lq  :1.33
# L   :2.00
# W   :0.50
# Wq  :0.33

==================MM1 =======================
p0* :0.33333
p1* :0.22222
Lq  :1.33
L   :2.00
W   :0.50
Wq  :0.33


# M/M/1/K system
$$\Bigg\{
    \begin{array}{ll}
    \frac{1-\rho}{1-\rho^{K+1}} & \rho\neq 1\\
    \frac{1}{K+1} & \rho=1
    \end{array}
$$

$$
L=\frac{\rho}{1-\rho}=\frac{\lambda}{\mu-\lambda}=  \frac{\rho \pi_0}{(1-\rho)^2}
$$

In [4]:
#Code503.py

#from Code501 import Queueing

# p0
def getP0(**p):
    rho, k = p['rho1'], p['k']
    return (1 - rho)/(1 - rho**(k+1))

# Lq
def getLq(**p):
    rho, k, p0 = p['rho1'], p['k'], p['p0']    #
    return rho/(1-rho) * ( 1 - p0 * (k*rho**k + 1))

#
laws ={ 'MM1K' : { 'p0' : getP0, 'Lq' : getLq, }}

#=========================================================================
# Tests
def getMM1K(mu, lamda, K):
    qs = Queueing(
            model = 'MM1K', 
            A ={'D':'Pois' , 'params': { 'lambda': lamda}},
            B ={'D':'Expo' , 'params': { 'mu': mu}},
            C = 1,
            K =  K,
            laws=laws)
    qs.test()

if __name__ == "__main__": getMM1K(10, 5, 12)   
# ==================MMS =======================
# p0* :0.50006
# p1* :0.25003
# Lq  :0.50
# L   :1.00
# W   :0.20
# Wq  :0.10

==================MM1K =======================
p0* :0.50006
p1* :0.25003
Lq  :0.50
L   :1.00
W   :0.20
Wq  :0.10


# M/M/s system
$$
\pi_0 = \left(  \sum_{n=0}^{s-1} \frac{s^n}{n!} \rho^{n} +  \frac{s^s}{s!} \rho^{s} \frac{1}{1 - \rho}  \right)^{-1}
$$

$$
L=\frac{\rho}{1-\rho}=\frac{\lambda}{\mu-\lambda}=  \frac{\rho \pi_0}{(1-\rho)^2}
$$

In [5]:
#Code504.py
#from Code501 import Queueing
#from math import factorial

#
fsum01 = lambda rho,s, n : 1 if n == 0 else fsum01(rho,s, n-1) + (s**n/factorial(n))*rho**n

# p0 :
def getP0(**p):
    rho,s = p['rho1'], p['s']
    return 1/( fsum01(rho,s , s-1) + (s**s/factorial(s)) * rho**s * (1/(1 - rho)))

# Lq
def getLq(**p):
    rho, s, p0 = p['rho1'], p['s'], p['p0']
    return (s**s /factorial(s)) * p0 * rho**(s+1)/(1-rho)**2

#
laws ={ 'MMS' : { 'p0' : getP0,  'Lq' : getLq }}

#=========================================================================
# Tests
def getMMS(mu, lamda, S):
    qs = Queueing(
            model = 'MMS', 
            A ={'D':'Pois' , 'params': { 'lambda': lamda}},
            B ={'D':'Expo' , 'params': { 'mu': mu}},
            C = S,
            laws=laws)
    qs.test()

if __name__ == "__main__":  getMMS(5,10,4)
# ==================MMS =======================
# p0* :0.13043
# p1* :0.26087
# Lq  :0.17
# L   :2.17
# W   :0.22
# Wq  :0.02

==================MMS =======================
p0* :0.13043
p1* :0.26087
Lq  :0.17
L   :2.17
W   :0.22
Wq  :0.02


# M/M/s/K system

$$\pi_0 = \left\{
\begin{array}{ll}
    \left( \sum_{n=0}^{s-1} \frac{s^n}{n!} \rho^{n} +  \frac{s^s}{s!} \rho^{s} \frac{1 - \rho^{K-s+1}}{1 - \rho} \right)^{-1} & \rho\neq 1  \\
    \left(\sum_{n=0}^{s} \frac{s^n}{n!}  +  \frac{s^s}{s!}  (K-s) \right)^{-1} &  \rho=1
\end{array}
\right.
$$

$$
L_q = \left\{
        \begin{array}{ll}
        \frac{s^s}{s!} \pi_0 \frac{\rho^{s+1}}{(1 - \rho)^2} ( 1 +(K-s)\rho^{K-s+1} - (K-s+1)\rho^{K-s} ) \qquad \rho \neq 1\\
        \frac{s^s}{s!} \pi_0 \frac{(K-s+1)(K-s)}{2} \qquad \rho=1
        \end{array}
\right.
$$

In [6]:
#Code505.py
#from Code501 import Queueing
#from math import factorial

#
fsum = lambda rho,s, n : 1 if n == 0 else fsum(rho,s, n-1) + (s**n/factorial(n))*rho**n

# p0
def getP0(**p):
    rho, s, k = p['rho1'], p['s'], p['k']
    return 1/( fsum(rho,s , s-1)+((s**s/factorial(s))*rho**s * (1/(1 - rho)))*(1 - rho**(k-s+1)))

# Lq
def getLq(**p):
    rho, s, k, p0 = p['rho1'], p['s'], p['k'], p['p0']
    return (s**s /factorial(s))* rho**(s+1)*p0*((1+(k-s)*rho**(k-s+1)- \
           (k-s+1)*rho**(k-s))/(1 - rho)**2)

#
laws ={ 'MMSK' : { 'p0' : getP0, 'Lq' : getLq}}

#=========================================================================
# Tests
def getMMSK(mu, lamda, S, K):
    qs = Queueing(
            model = 'MMSK', 
            A ={'D':'Pois' , 'params': { 'lambda': lamda}},
            B ={'D':'Expo' , 'params': { 'mu': mu}},
            C = S,
            K = K,
            laws=laws)
    qs.test()

if __name__ == "__main__": getMMSK(4, 5, 2, 15)

# ==================MMSK =======================
# p0* :0.23092
# p1* :0.28865
# Lq  :0.79
# L   :2.04
# W   :0.41
# Wq  :0.16

==================MMSK =======================
p0* :0.23092
p1* :0.28865
Lq  :0.79
L   :2.04
W   :0.41
Wq  :0.16
